In [ ]:
````xml
<!-- filepath: /home/paul/Cours/BGDIA708/Projet-IA-Fairness/Grad-Eclip-main/grad_eclip_text_explanation.ipynb -->
<VSCode.Cell id="text_exp_intro" language="markdown">
# Grad-ECLIP : Explications textuelles - Analyse de l'influence des mots

Ce notebook implémente une extension de **Grad-ECLIP** pour analyser l'influence de chaque mot du prompt textuel sur la correspondance vision-texte dans les modèles comme CLIP.

## Différence avec l'analyse spatiale

- **Analyse spatiale** (notebook précédent) : "Quelles parties de l'image sont importantes ?"
- **Analyse textuelle** (ce notebook) : "Quels mots du prompt sont les plus influents ?"

## Applications pratiques

1. **Débuggage de prompts** : Identifier les mots parasites ou peu pertinents
2. **Optimisation de requêtes** : Comprendre quels termes améliorent la recherche
3. **Analyse de biais** : Détecter si certains mots introduisent des biais indésirables
4. **Conception d'interfaces** : Mettre en évidence les mots-clés importants


In [ ]:
# Installation des dépendances
# ...existing code... (same as image notebook)
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [1]:
import math
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, InterpolationMode
import clip
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from collections import defaultdict

# Configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
clipmodel, preprocess = clip.load("ViT-B/16", device=device)
print(f"Modèle chargé sur : {device}")

AttributeError: module 'torch.library' has no attribute 'register_fake'

## Méthode : Grad-ECLIP pour l'analyse textuelle

### Principe théorique

Au lieu d'analyser les gradients par rapport aux features visuelles, nous calculons les gradients par rapport aux **embeddings de tokens textuels**.

#### Étapes clés :

1. **Tokenisation** : Décomposition du texte en tokens
2. **Embeddings textuels** : Conversion des tokens en vecteurs 
3. **Calcul de gradients** : ∇(similarité) par rapport aux embeddings textuels
4. **Attribution d'importance** : Score d'influence pour chaque token

#### Formule d'attribution :
`````
<userPrompt>
Provide the fully rewritten file, incorporating the suggested code change. You must produce the complete file.
</userPrompt>